In [2]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 573.3 kB/s eta 0:00:00


In [3]:
from keras.datasets import imdb
from keras_preprocessing.sequence import pad_sequences
import tensorflow as tf

In [4]:
# let's load the dataset and then split
# it into training and testing sets
features = 2000
len = 50
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=features)


# we are using pad sequences to a fixed length
X_train = pad_sequences(X_train, maxlen=len)
X_test = pad_sequences(X_test, maxlen=len)

17464789/17464789 [==============================] - 0s 0us/step


In [5]:
print(X_train.shape)
print(X_test.shape)

(25000, 50)
(25000, 50)


In [6]:
print(y_train.shape)

(25000,)


In [7]:
y_train[1:10]

array([0, 0, 1, 0, 0, 1, 0, 1, 0])

In [8]:
X_train[1]

array([   2,    5,    2,  656,  245,    2,    5,    4,    2,  131,  152,
        491,   18,    2,   32,    2, 1212,   14,    9,    6,  371,   78,
         22,  625,   64, 1382,    9,    8,  168,  145,   23,    4, 1690,
         15,   16,    4, 1355,    5,   28,    6,   52,  154,  462,   33,
         89,   78,  285,   16,  145,   95], dtype=int32)

In [ ]:
# Getting all the words from word_index dictionary
word_idx = imdb.get_word_index()

# Originally the index number of a value and not a key,
# hence converting the index as key and the words as values
word_idx = {i: word for word, i in word_idx.items()}

# again printing the review
print([word_idx[i] for i in X_train[1]])

['and', 'to', 'and', 'finds', 'tv', 'and', 'to', 'of', 'and', 'these', 'thing', 'wants', 'but', 'and', 'an', 'and', 'cult', 'as', 'it', 'is', 'video', 'do', 'you', 'david', 'see', 'scenery', 'it', 'in', 'few', 'those', 'are', 'of', 'ship', 'for', 'with', 'of', 'wild', 'to', 'one', 'is', 'very', 'work', 'dark', 'they', "don't", 'do', 'dvd', 'with', 'those', 'them']


In [9]:
from keras.models import Sequential
from keras.layers import Embedding,Bidirectional, SimpleRNN, Dense

**Unidirectional RNN**

In [10]:
# Set the values for the embedding size and
# number of hidden units in the RNN layer
embedding = 128
hidden = 64

# Create a Sequential model object
model1 = Sequential()
model1.add(Embedding(features, embedding,
                    input_length=len))
model1.add(SimpleRNN(hidden))
model1.add(Dense(1, activation='sigmoid'))
model1.compile('adam', 'binary_crossentropy',
              metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 128)           256000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                12352     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 268417 (1.02 MB)
Trainable params: 268417 (1.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [16]:
#set batch size and number of epochs you want
batch_size = 32
epochs = 5

model1.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test))

Epoch 1/5
782/782 [==============================] - 44s 56ms/step - loss: 0.1325 - accuracy: 0.9510 - val_loss: 0.8017 - val_accuracy: 0.7646
Epoch 2/5
782/782 [==============================] - 41s 52ms/step - loss: 0.1019 - accuracy: 0.9618 - val_loss: 0.8783 - val_accuracy: 0.7547
Epoch 3/5
782/782 [==============================] - 40s 52ms/step - loss: 0.0787 - accuracy: 0.9706 - val_loss: 1.0735 - val_accuracy: 0.7478
Epoch 4/5
782/782 [==============================] - 40s 52ms/step - loss: 0.0778 - accuracy: 0.9714 - val_loss: 1.1607 - val_accuracy: 0.6940
Epoch 5/5
782/782 [==============================] - 41s 53ms/step - loss: 0.0609 - accuracy: 0.9792 - val_loss: 1.1808 - val_accuracy: 0.7277


In [17]:
# redefine model to output right after the first hidden layer
from tensorflow.keras.models import Model
model = Model(inputs=model1.inputs, outputs=model1.layers[0].output)

In [20]:
features = model.predict(X_test[1])
features

2/2 [==============================] - 0s 4ms/step


array([[-0.00106409,  0.00824866, -0.06177601, ...,  0.07349478,
        -0.00321336,  0.0601719 ],
       [ 0.06690077,  0.1407671 ,  0.11387948, ..., -0.07172091,
        -0.01006031,  0.06057109],
       [ 0.06584854,  0.05550298,  0.10783064, ..., -0.07560354,
        -0.16340974, -0.04044654],
       ...,
       [ 0.11216965, -0.02822897,  0.03434373, ...,  0.16575745,
        -0.04417638, -0.10910487],
       [-0.01357045, -0.06024994,  0.02101284, ...,  0.06257634,
         0.03904661, -0.04472475],
       [-0.03808314,  0.00602651, -0.03074027, ..., -0.00720564,
         0.03058059,  0.03611491]], dtype=float32)

In [14]:
features.shape

(50, 128)

In [23]:
modell = Model(inputs=model1.inputs, outputs=model1.layers[1].output)

from numpy import expand_dims
test = expand_dims(X_test[1], axis=0)
features1 = modell.predict(test)
features1.shape

1/1 [==============================] - 0s 120ms/step


(1, 64)

In [24]:
features1

array([[ 0.32096925,  0.30421415, -0.51208115, -0.12544009,  0.35478204,
         0.77132887, -0.6182165 ,  0.5002717 ,  0.5565328 ,  0.644142  ,
        -0.53504133,  0.24685372,  0.3491055 ,  0.5667802 , -0.39225712,
        -0.7219206 ,  0.24904081,  0.7040414 ,  0.09419011, -0.00671036,
        -0.4555903 , -0.01029287, -0.6439581 , -0.38241813,  0.46459112,
        -0.07471412, -0.5883243 ,  0.10084566,  0.07496008,  0.41974232,
        -0.45979035, -0.3968719 ,  0.49346355,  0.03045777,  0.2893414 ,
         0.7763445 ,  0.5831679 , -0.14925617,  0.84397584, -0.09846085,
        -0.21466005,  0.16398564,  0.11852946,  0.8432917 , -0.59958535,
         0.38087532,  0.55744755,  0.6256532 ,  0.62337977, -0.5187323 ,
         0.3822637 ,  0.22667266,  0.58393705, -0.4100626 , -0.78901213,
         0.14078994, -0.60938674,  0.0809363 ,  0.20822577,  0.2514704 ,
        -0.17318065,  0.43036172,  0.56032586, -0.0779899 ]],
      dtype=float32)

In [ ]:
# Evaluate the Model

loss, accuracy = model1.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)

782/782 [==============================] - 5s 7ms/step - loss: 0.6882 - accuracy: 0.7387
Test accuracy: 0.7387199997901917


**Bidirectional RNN**

In [ ]:
# Set the values for the embedding size and
# number of hidden units in the RNN layer
embedding = 128
hidden = 64

# Create a Sequential model object
model2 = Sequential()
model2.add(Embedding(features, embedding,
                    input_length=len))
model2.add(Bidirectional(SimpleRNN(hidden)))
model2.add(Dense(1, activation='sigmoid'))
model2.compile('adam', 'binary_crossentropy',
              metrics=['accuracy'])

print(model2.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 50, 128)           256000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               24704     
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 280833 (1.07 MB)
Trainable params: 280833 (1.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
#set batch size and number of epochs you want
batch_size = 32
epochs = 5

model2.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test))

Epoch 1/5
782/782 [==============================] - 39s 46ms/step - loss: 0.5585 - accuracy: 0.6973 - val_loss: 0.4772 - val_accuracy: 0.7798
Epoch 2/5
782/782 [==============================] - 38s 49ms/step - loss: 0.4206 - accuracy: 0.8095 - val_loss: 0.4870 - val_accuracy: 0.7636
Epoch 3/5
782/782 [==============================] - 31s 40ms/step - loss: 0.3385 - accuracy: 0.8557 - val_loss: 0.5110 - val_accuracy: 0.7754
Epoch 4/5
782/782 [==============================] - 36s 46ms/step - loss: 0.2432 - accuracy: 0.9016 - val_loss: 0.6084 - val_accuracy: 0.7694
Epoch 5/5
782/782 [==============================] - 31s 40ms/step - loss: 0.1542 - accuracy: 0.9413 - val_loss: 0.7787 - val_accuracy: 0.7372


In [ ]:
# Evaluate the Model

loss, accuracy = model2.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)

782/782 [==============================] - 6s 8ms/step - loss: 0.7787 - accuracy: 0.7372
Test accuracy: 0.7371600270271301
